In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
import random
import numpy as np
from model import ClassifierNet
import torch
from dataset import ItemsDataModule
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

torch.manual_seed(42)
random.seed(42)
np.random.seed(42)


In [3]:
import os

os.listdir("kfold_models")


['kfold_4_08673.ckpt',
 'kfold_2_08663.ckpt',
 'kfold_3_08646.ckpt',
 'kfold_0_08651.ckpt',
 'kfold_1_08661.ckpt']

In [4]:
default_net = ClassifierNet(
    {"num_classes": 730},
)

dm = ItemsDataModule(batch_size=4, num_workers=20)
dm.setup()


kfold_preds = []
for file in os.listdir("kfold_models"):
    default_net = default_net.load_from_checkpoint(f"kfold_models/{file}")

    trainer = Trainer(
        accelerator="gpu",
        devices=[0],
    )

    p = trainer.predict(default_net, dm.test_dataloader())
    kfold_preds.append(p)


Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.

Predicting: 0it [00:00, ?it/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

Predicting: 0it [00:00, ?it/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

Predicting: 0it [00:00, ?it/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

Predicting: 0it [00:00, ?it/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

Predicting: 0it [00:00, ?it/s]

In [5]:
preds_all = []
probs_all = []

for p in kfold_preds:
    preds = torch.cat([i[0] for i in p])
    probs = torch.cat([i[4] for i in p])
    preds_all.append(preds)
    probs_all.append(probs)


In [6]:
preds_all = torch.stack(preds_all).T
probs_all = torch.stack(probs_all).transpose(0, 1).transpose(1, 2)
preds_all.shape, probs_all.shape


(torch.Size([16860, 5]), torch.Size([16860, 730, 5]))

In [7]:
import pandas as pd

train_df = pd.read_parquet("preprocessed_train.parquet")
test_df = pd.read_parquet("preprocessed_test.parquet")


In [8]:
counter = 0
for i, (p, probs) in enumerate(zip(preds_all, probs_all)):
    if not (p == p[0]).all():
        counter += 1
counter


3215

Видим что в целом 3к из 16к емплов неоднозначностей в результатах ансамбля. Что неплохо

Посмотрим некоторые примеры 

In [9]:
for i, (p, probs) in enumerate(zip(preds_all, probs_all)):
    if not (p == p[0]).all():
        print(test_df.iloc[i]["product_id"])
        print(probs.max(dim=0))
        print(test_df.iloc[i]["title"])
        print("----------------------------------------------------------------------")
    if i > 10:
        break


1997646
torch.return_types.max(
values=tensor([0.9350, 0.5117, 0.9934, 0.4364, 0.5368]),
indices=tensor([453, 453, 453, 428, 206]))
Светодиодная лента Smart led Strip Light, с пультом, метров, USB, Bluetooth
----------------------------------------------------------------------
1668662
torch.return_types.max(
values=tensor([0.8276, 0.9919, 0.9997, 0.6497, 0.9994]),
indices=tensor([181, 438, 438, 302, 181]))
Декоративная табличка Правила кухни подставка под горячее, разделочная доска
----------------------------------------------------------------------
1467778
torch.return_types.max(
values=tensor([0.9154, 0.9953, 0.9976, 0.9958, 0.9218]),
indices=tensor([242, 440, 440, 440, 440]))
Подставка под ложку керамическая, подложка Клубника Лаванда Лимон
----------------------------------------------------------------------
342378
torch.return_types.max(
values=tensor([0.6378, 0.8902, 0.5955, 0.8247, 0.6013]),
indices=tensor([581,  65,  65, 581, 581]))
Топ Чебоксарский трикотаж
---------------

In [10]:
train_df[train_df["target"] == 181].iloc[0][["title", "category_name"]].tolist()


['Разделочные подарочные доски с Новогодней тематикой',
 'Все категории->Товары для дома->Товары для кухни->Кухонные инструменты->Разделочные доски']

In [11]:
train_df[train_df["target"] == 438].iloc[0][["title", "category_name"]].tolist()


['Крючок без сверления для фотографий, картин, постеров, (Крючки для твердых стен компл. )',
 'Все категории->Товары для дома->Декор и интерьер->Таблички, номера и крючки']

Видим, что есть примеры товаров, где трудно самому определить тип. (Пример то ли разделочная доска, то ли табличка)

In [80]:
target_to_category_id = (
    train_df[["category_id", "target"]]
    .drop_duplicates()
    .set_index("target", drop=True)
    .to_dict()["category_id"]
)
len(target_to_category_id)


730

In [128]:
final_preds = []

for i, (p, probs) in enumerate(zip(preds_all, probs_all)):
    if not (p == p[0]).all():
        max_probs = probs.max(dim=0).values

        counts = p.unique(return_counts=True)
        if (counts[1] >= 3).any() or (counts[1] == 2).sum() == 1:
            idx = counts[1].argmax()
            pred = int(counts[0][idx])

        elif (counts[1] == 2).sum() == 2:
            top_idxs = (counts[1] == 2).nonzero(as_tuple=True)[0]
            top_class_1 = counts[0][top_idxs[0]]
            top_class_2 = counts[0][top_idxs[1]]

            neural_nets_1 = (p == top_class_1).nonzero(as_tuple=True)[0]
            neural_nets_2 = (p == top_class_2).nonzero(as_tuple=True)[0]

            sum_probs_1 = max_probs[neural_nets_1].sum()
            sum_probs_2 = max_probs[neural_nets_2].sum()

            pred = int(top_class_1) if sum_probs_1 > sum_probs_2 else int(top_class_2)
        else:
            max_prob_idx = max_probs.argmax()
            pred = int(p[max_prob_idx])

        final_preds.append(pred)

    else:
        final_preds.append(int(p[0]))
len(final_preds)


16860

In [130]:
category_preds = [target_to_category_id[i] for i in final_preds]
len(category_preds), min(category_preds)


(16860, 2599)

In [141]:
for i in category_preds:
    if i not in train_df["category_id"].unique():
        print(i)


In [143]:
test_df["predicted_category_id"] = category_preds


In [157]:
test_df.iloc[100][["title", "predicted_category_id"]].tolist()


['Чехол книжка для Honor', 11937]

In [158]:
train_df[train_df["category_id"] == 11937].iloc[0]["category_name"]


'Все категории->Электроника->Смартфоны и телефоны->Аксессуары и запчасти->Чехлы'

Похоже на правду :) 

Удаляем лишние столбцы и коммитим

In [160]:
columns_to_drop = test_df.columns.tolist()
columns_to_drop.remove("product_id")
columns_to_drop.remove("predicted_category_id")
result_df = test_df.drop(columns=columns_to_drop)
result_df


,product_id,predicted_category_id
0,1997646,12919
1,927375,14922
2,1921513,2803
3,1668662,13755
4,1467778,13887
...,...,...
16855,1914264,11645
16856,1310569,12357
16857,978095,13651
16858,797547,2740


In [161]:
result_df.to_parquet("result.parquet")


In [13]:
pd.read_parquet("result.parquet").columns


Index(['product_id', 'predicted_category_id'], dtype='object')